In [24]:
!nvidia-smi

Sat Jun  1 13:40:15 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:01:00.0 Off |                  N/A |
|  0%   49C    P8              12W / 170W |  12029MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
!nvidia-smi

Thu Jun  6 08:55:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060        Off | 00000000:01:00.0 Off |                  N/A |
|  0%   36C    P8               5W / 170W |      8MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from bark import SAMPLE_RATE, generate_audio, preload_models
from bark.api import semantic_to_waveform
from bark.generation import (
    generate_text_semantic,
    preload_models,
)

from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
import nltk
import numpy as np

nltk.download('punkt')

# download and load all models
preload_models()

/home/thoth/anaconda3/envs/diffuse/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
import os

book_filepath = "./data/greatgodpan_simplified.txt"

file = open(book_filepath, "r")
book_text = file.read()
file.close()
type(book_text)

str

In [4]:
print(book_text)

                           The Great God Pan by Arthur Machen

I
THE EXPERIMENT

   “I am glad you came, Clarke; very glad indeed. I was not sure
   you could spare the time.”

   “I was able to make arrangements for a few days; things are not
   very lively just now. But have you no misgivings, Raymond? Is
   it absolutely safe?”

   The two men were slowly pacing the terrace in front of Dr.
   Raymond’s house. The sun still hung above the western
   mountain-line, but it shone with a dull red glow that cast no
   shadows, and all the air was quiet; a sweet breath came from
   the great wood on the hillside above, and with it, at
   intervals, the soft murmuring call of the wild doves. Below, in
   the long lovely valley, the river wound in and out between the
   lonely hills, and, as the sun hovered and vanished into the
   west, a faint mist, pure white, began to rise from the hills.
   Dr. Raymond turned sharply to his friend.

   “Safe? Of course it is. In itself the operation is 

In [ ]:
###modifications I want
#pause after character ends talking
#pause after paragraph break
#different voices for different characters

In [5]:
sentences = nltk.sent_tokenize(book_text.replace("\n", " ").strip())

[nltk_data] Downloading package punkt to /home/thoth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
len(sentences)

833

In [8]:
sentences[0]

'The Great God Pan by Arthur Machen  I THE EXPERIMENT     “I am glad you came, Clarke; very glad indeed.'

In [6]:
SPEAKER = "v2/en_speaker_6"
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence #1:45 #4:00, 2 ish hours. Don't know if it all completed though

pieces = []
for sentence in sentences:
    audio_array = generate_audio(sentence, history_prompt=SPEAKER)
    pieces += [audio_array, silence.copy()]

100%|███████████████████████████████████████████| 35/35 [00:18<00:00,  1.89it/s]
/home/thoth/anaconda3/envs/diffuse/lib/python3.9/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
100%|███████████████████████████████████████████| 38/38 [00:18<00:00,  2.03it/s]


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_SUPPORTED when calling `cublasGemmEx( handle, opa, opb, m, n, k, &falpha, a, CUDA_R_16BF, lda, b, CUDA_R_16BF, ldb, &fbeta, c, CUDA_R_16BF, ldc, compute_type, CUBLAS_GEMM_DEFAULT_TENSOR_OP)`

In [7]:
result = np.concatenate(pieces)
write_wav("text_longer_test_audio_file.wav", SAMPLE_RATE, result)

In [16]:
sentences[0:10]

['The Great God Pan by Arthur Machen  I THE EXPERIMENT     “I am glad you came, Clarke; very glad indeed.',
 'I was not sure    you could spare the time.”     “I was able to make arrangements for a few days; things are not    very lively just now.',
 'But have you no misgivings, Raymond?',
 'Is    it absolutely safe?”     The two men were slowly pacing the terrace in front of Dr.    Raymond’s house.',
 'The sun still hung above the western    mountain-line, but it shone with a dull red glow that cast no    shadows, and all the air was quiet; a sweet breath came from    the great wood on the hillside above, and with it, at    intervals, the soft murmuring call of the wild doves.',
 'Below, in    the long lovely valley, the river wound in and out between the    lonely hills, and, as the sun hovered and vanished into the    west, a faint mist, pure white, began to rise from the hills.',
 'Dr. Raymond turned sharply to his friend.',
 '“Safe?',
 'Of course it is.',
 'In itself the operation

In [20]:
SPEAKER = "v2/en_speaker_5"
GEN_TEMP = 0.6
silence = np.zeros(int(0.25 * SAMPLE_RATE))  # quarter second of silence
num_pieces_run = 0

pieces = []
for sentence in sentences[0:10]:

    semantic_tokens = generate_text_semantic(
        sentence,
        history_prompt=SPEAKER,
        temp=GEN_TEMP,
        min_eos_p=0.01,  # this controls how likely the generation is to end
    )
    
    audio_array = semantic_to_waveform(semantic_tokens, history_prompt=SPEAKER,)    
    pieces += [audio_array, silence.copy()]
    num_pieces_run+=1

100%|███████████████████████████████████████████| 29/29 [00:15<00:00,  1.88it/s]
/home/thoth/anaconda3/envs/diffuse/lib/python3.9/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
100%|███████████████████████████████████████████| 37/37 [00:18<00:00,  1.97it/s]


In [21]:
num_pieces_run

10

In [22]:
result = np.concatenate(pieces)
write_wav("text_longer_test_audio_file2.wav", SAMPLE_RATE, result)

In [ ]:
Audio(np.concatenate(pieces), rate=SAMPLE_RATE)